# Certificates (X.509)

Import useful packages for project

In [79]:
# For a CSR, self-signed and key type

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.serialization import load_pem_private_key
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes

import OpenSSL.crypto
from OpenSSL.crypto import load_certificate_request, FILETYPE_PEM

## Create a Certificate Signing Request (CSR)

**Steps for request a signing certificate**

1. Generate a private/public key pair.
2. Create a request for a certificate, which is signed by our key (to prove that we own that key).
3. Give our CSR to a CA (but not the private key).
4. The CA validates that we own the resource (e.g. domain) we want a certificate for.
5. The CA gives us a certificate, signed by them, which identifies our public key, and the resource we are authenticated for.
6. We configure our server to use that certificate, combined with our private key, to server traffic.

### First step

In [75]:
# Generate our key

try:
    with open('key.pem', 'rb') as f:
        key_data = f.read()
    key = load_pem_private_key(key_data, b"password", default_backend())
except FileNotFoundError:
    print("error")
    key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    with open('key.pem', 'wb+') as f:
        f.write(key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.BestAvailableEncryption(b"password")
        ))
finally:
    print(key)

### Second step

In [77]:
# Generate a CSR

csr = x509.CertificateSigningRequestBuilder().subject_name(x509.Name([
    # Provide various details about who we are.
    x509.NameAttribute(NameOID.COUNTRY_NAME, u"CH"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"Neuchatel"),
    x509.NameAttribute(NameOID.LOCALITY_NAME, u"Neuchatel"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"HE-Arc"),
    x509.NameAttribute(NameOID.COMMON_NAME, u"www.he-arc.ch"),
])).add_extension(
    x509.SubjectAlternativeName([
        # Describe what sites we want this certificate for.
        x509.DNSName(u"www.he-arc.ch"),
        x509.DNSName(u"webmail.he-arc.ch"),
        x509.DNSName(u"intranet.he-arc.ch"),
    ]),
critical=False,
# Sign the CSR with our private key.
).sign(key, hashes.SHA256(), default_backend())

# Write our CSR out to disk.

with open("csr.pem", "wb") as f:
    f.write(csr.public_bytes(serialization.Encoding.PEM))

### Read and deserialize the CSR

In [99]:
with open("csr.pem", "rb") as f:
    csr = f.read()
print(str(csr, 'utf-8'))

-----BEGIN CERTIFICATE REQUEST-----
MIIC9TCCAd0CAQAwXjELMAkGA1UEBhMCQ0gxEjAQBgNVBAgMCU5ldWNoYXRlbDES
MBAGA1UEBwwJTmV1Y2hhdGVsMQ8wDQYDVQQKDAZIRS1BcmMxFjAUBgNVBAMMDXd3
dy5oZS1hcmMuY2gwggEiMA0GCSqGSIb3DQEBAQUAA4IBDwAwggEKAoIBAQCYzSxR
GdDsHB8y05+30/8frz91pAIBojU58knJQFZMxODji1DpziBYYs76xr0GRx8Z7Gls
2RT5NDdc72LK4ADHNYKV/8eGvLUH2h7jlm/BuP/VorNH4DoblKx4vZIKzhngRVN9
lBY0zFaapjUDEahUPUywWu/CxrgTKVzaOFqRowHfjLouM0n99Csx0VNd+ZXXR6Fc
C5jcB/SaKbCaF5PUX2jc6Uq88QfwJqyAJ8NICCgenWz4XXN0R7OdfoawEuWW5Z9M
1NrdEN/Ho3IBgc3kmSsyuphrB2iMXhFLKhEJMmBgqJIfF3QICwezW54BJBV6IWL6
ClfhOSoYMXLrvEadAgMBAAGgUjBQBgkqhkiG9w0BCQ4xQzBBMD8GA1UdEQQ4MDaC
DXd3dy5oZS1hcmMuY2iCEXdlYm1haWwuaGUtYXJjLmNoghJpbnRyYW5ldC5oZS1h
cmMuY2gwDQYJKoZIhvcNAQELBQADggEBAAqX+z9x3Y17ObMdG4qqzACLytBYFb4Q
nGI/GB4a/Tv5OJQOISEbkh3IZ1LYf/3yGHxSaJho65mwfnPrj0/0CLjTmVsfwTFB
g5tQBJLItdscbyt0RKlIDk+qPxWT1TZPqtAAj0px8CsoBQOZSr6tTLSoz+IfEjeK
1y5XSOTeEZBCvRoSILWzqb/Qbxz0O+7evtOYtfbl4cXpK/vqu/33B3VxyNvIGq49
irtnvduJieMpGeu4BZlANon1JgfWJk+prKApMbT2k/AmCjlVdxvXNn

In [116]:
req = load_certificate_request(FILETYPE_PEM, csr)
key = req.get_pubkey()

key_type = 'RSA' if key.type() == OpenSSL.crypto.TYPE_RSA else 'DSA'

subject = req.get_subject()
extensions = req.get_extensions()

components = dict(subject.get_components())

print("Country :", str(components[b'C'], 'utf-8'))
print("State :", str(components[b'ST'], 'utf-8'))
print("City :", str(components[b'L'], 'utf-8'))
print("Company :", str(components[b'O'], 'utf-8'))
print("Website :", str(components[b'CN'], 'utf-8'))

print(extensions[0])

Country : CH
State : Neuchatel
City : Neuchatel
Company : HE-Arc
Website : www.he-arc.ch
DNS:www.he-arc.ch, DNS:webmail.he-arc.ch, DNS:intranet.he-arc.ch


## Creating a self-signed certificate

## Determining Certificate or Certificate Signing Resquest Key Type